In [9]:
import pandas as pd
from IPython.display import HTML
import re
from itertools import product
import random
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor

In [10]:
df_type = pd.read_csv('../../data/preprocessed_data/시도_시군구별_보행자_사고_사고유형_전처리ver.csv')
df_type.drop(['법정동코드', '사고내용', '피해자 상해정도'], axis=1, inplace=True)
# df_type
HTML(df_type.head(5).to_html())

,연도,시군구,사망자수,중상자수,경상자수,부상신고자수,사고유형,도로형태
0,2020,서울특별시 종로구 창성동,0,1,0,0,차대사람 - 차도통행중,교차로 - 교차로부근
1,2020,서울특별시 종로구 창성동,0,1,0,0,차대사람 - 차도통행중,교차로 - 교차로안
2,2020,서울특별시 종로구 통인동,0,0,1,0,차대사람 - 기타,단일로 - 기타
3,2020,서울특별시 종로구 누상동,0,0,1,0,차대사람 - 차도통행중,교차로 - 교차로안
4,2020,서울특별시 종로구 누상동,0,0,1,0,차대사람 - 기타,단일로 - 기타


In [11]:
df_time = pd.read_excel('../../data/preprocessed_data/시도_시군구_시간대별_노인교통사고_전처리ver.xlsx', header=[0,1])
# df_time
HTML(df_time.head(5).to_html())

In [12]:
# 사고건수 데이터만 필터링
df_accident = df_time[df_time[('기준년도', '기준년도')] == '사고[건]'].copy()

# 다중 컬럼 > 단일 문자열 컬럼
df_accident.columns = ['{}_{}'.format(y, x) if isinstance(x, str) else str(y) for y, x in df_accident.columns]

# id 변수
id_vars = ['시도_시도', '시군구_시군구', '기준년도_기준년도']

# 시간대 컬럼만 추출
value_vars = [col for col in df_accident.columns if re.match(r'^\d{4}_\d{1,2}시~\d{1,2}시$', col)]

# wide > long 변환
df_melt = df_accident.melt(id_vars=id_vars, value_vars=value_vars,
                           var_name='연도_시간대', value_name='사고건수')

# 연도, 시간대 분리
df_melt[['연도', '시간대']] = df_melt['연도_시간대'].str.extract(r'(\d{4})_(.+)')

# 컬럼명 정리
df_melt = df_melt.rename(columns={
    '시도_시도': '시도',
    '시군구_시군구': '시군구',
    '기준년도_기준년도': '기준년도'
})

# 사고건수 및 사고여부 처리
df_melt['사고건수'] = pd.to_numeric(df_melt['사고건수'], errors='coerce').fillna(0).astype(int)
df_melt['사고여부'] = (df_melt['사고건수'] > 0).astype(int)

# 사고건수 제거 > 모델 학습용 테이블
df_model = df_melt[['연도', '시도', '시군구', '시간대', '사고여부']].copy()
df_model['사고여부'] = df_model['사고여부'].astype(str)
df_model

# 위험도로 계산

,연도,시도,시군구,시간대,사고여부
0,2020,서울,합계,0시~2시,1
1,2020,서울,종로구,0시~2시,1
2,2020,서울,중구,0시~2시,1
3,2020,서울,용산구,0시~2시,1
4,2020,서울,성동구,0시~2시,1
...,...,...,...,...,...
14815,2024,경남,합천군,22시~24시,0
14816,2024,경남,창원시(통합),22시~24시,1
14817,2024,제주,합계,22시~24시,1
14818,2024,제주,제주시,22시~24시,1


In [13]:
# 양성 샘플 키 추출
positive_keys = set(df_model[df_model['사고여부'] == '1'][['연도', '시도', '시군구', '시간대']].apply(tuple, axis=1))

# 전체 가능한 조합 생성
years = df_model['연도'].unique()
cities = df_model['시도'].unique()
districts = df_model['시군구'].unique()
timezones = df_model['시간대'].unique()

all_possible = set(product(years, cities, districts, timezones))

# 음성 샘플 후보 = 사고 없는 조합
negative_keys = list(all_possible - positive_keys)

# 양성 샘플 수에 맞춰 음성 샘플링
num_positive = len(positive_keys)
random.seed(42)
sampled_negatives = random.sample(negative_keys, min(num_positive, len(negative_keys)))

# 음성 샘플 DataFrame 생성
neg_df = pd.DataFrame(sampled_negatives, columns=['연도', '시도', '시군구', '시간대'])
neg_df['사고여부'] = '0'

# 양성 샘플 추출 및 병합
pos_df = df_model[df_model['사고여부'] == '1'].copy()
df_balanced = pd.concat([pos_df, neg_df], ignore_index=True)
df_balanced

,연도,시도,시군구,시간대,사고여부
0,2020,서울,합계,0시~2시,1
1,2020,서울,종로구,0시~2시,1
2,2020,서울,중구,0시~2시,1
3,2020,서울,용산구,0시~2시,1
4,2020,서울,성동구,0시~2시,1
...,...,...,...,...,...
26515,2022,경북,영월군,0시~2시,0
26516,2024,전북,서초구,22시~24시,0
26517,2021,광주,홍천군,2시~4시,0
26518,2023,경남,곡성군,2시~4시,0


In [14]:
# 타겟은 문자열로 설정
df_balanced['사고여부'] = df_balanced['사고여부'].astype(str)

# 학습/검증 분리
train_data, test_data = train_test_split(df_balanced, test_size=0.2, random_state=42, stratify=df_balanced['사고여부'])

# 모델 학습
predictor = TabularPredictor(label='사고여부', path="PredictAccidentOccur/").fit(train_data)

# 성능 평가
predictor.evaluate(test_data)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.4.0: Fri Apr 11 18:33:39 PDT 2025; root:xnu-11417.101.15~117/RELEASE_ARM64_T6020
CPU Count:          12
Memory Avail:       2.91 GB / 16.00 GB (18.2%)
Disk Space Avail:   24.80 GB / 460.43 GB (5.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='

{'accuracy': 0.9958521870286576,
 'balanced_accuracy': 0.9958521870286576,
 'mcc': 0.9917325763042215,
 'roc_auc': 0.9986205237403002,
 'f1': 0.9958677685950413,
 'precision': 0.9921407185628742,
 'recall': 0.9996229260935143}

In [16]:
leaderboard = predictor.leaderboard(silent=True)
print(leaderboard)

                  model  score_val eval_metric  pred_time_val  fit_time  \
0   WeightedEnsemble_L2   0.996230    accuracy       0.038979  1.477922   
1              CatBoost   0.995759    accuracy       0.001273  0.908944   
2       NeuralNetFastAI   0.995759    accuracy       0.011406  8.003872   
3            LightGBMXT   0.994345    accuracy       0.010126  1.925405   
4              LightGBM   0.993402    accuracy       0.006112  1.115939   
5         LightGBMLarge   0.992460    accuracy       0.010680  2.229755   
6      RandomForestGini   0.875118    accuracy       0.036283  0.478133   
7      RandomForestEntr   0.863808    accuracy       0.037409  0.528932   
8               XGBoost   0.860509    accuracy       0.009552  1.625571   
9        ExtraTreesGini   0.784637    accuracy       0.037150  0.396594   
10       ExtraTreesEntr   0.781810    accuracy       0.037565  0.414714   

    pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                 0.000297

In [15]:
# 예측
preds = predictor.predict(test_data)

# 저장
predictor.save("saved_model")

new_df = pd.DataFrame([{
    '연도': '2030',
    '시도': '서울',
    '시군구': '종로구',
    '시간대': '8시~10시'
}])

# 예측
predictor.predict(new_df)

0    1
Name: 사고여부, dtype: object